In [133]:
import pandas as pd

#Read the csv File
#Try Common Encodings: As a last resort, you can try common encodings like
#'utf-8', 'latin1', 'ISO-8859-1', etc., and see if any of them successfully decode the file without errors.

df = pd.read_csv('Bundesligadaten.txt', delimiter='\t', encoding='ISO-8859-1', header=None)

df.columns = ['date', 'time', 'localTeam', 'awayTeam', 'score', 'score-HT']


In [134]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [135]:
#df['date'].isnull().any()

In [136]:
for column in ('date', 'time'):
     # Check if the column contains NaN values
    if df[column].isnull().any():
        # Fill NaN values with corresponding values from the last row using "forward fill"
        df[column] = df[column].ffill()

In [137]:

# Split the date Value to create a Day of the Week       
df['DoW'] = df['date'].str.split(',').str[0]
df['date'] = df['date'].str.split(',').str[1]

# Split the score
split_values = df['score'].str.split(':')
df['localTeam-Score'] = split_values.str[0].str[-1]
df['awayTeam-Score'] = split_values.str[1].str[0]

# Split the score to half time
split_values = df['score-HT'].str.split(':')
df['localTeam-SHT'] = split_values.str[0].str[-1]
df['awayTeam-SHT'] = split_values.str[1].str[0]


# Reorder columns
df = df[['DoW', 'date', 'time', 'localTeam', 'awayTeam', 'score', 'localTeam-Score', 'awayTeam-Score', 'score-HT', 'localTeam-SHT', 'awayTeam-SHT']]

In [138]:
df.head(1)

,DoW,date,time,localTeam,awayTeam,score,localTeam-Score,awayTeam-Score,score-HT,localTeam-SHT,awayTeam-SHT
0,Fr,05.08.2022,20:30,Eintracht Frankfurt,- FC Bayern München,1:6,1,6,(0:5),0,5


In [139]:
# Triming Columns
df['score-HT'] = df['score-HT'].str.removeprefix('(').str.removesuffix(')')
df['awayTeam'] = df['awayTeam'].str.removeprefix('-')

In [140]:
df.head(1)

,DoW,date,time,localTeam,awayTeam,score,localTeam-Score,awayTeam-Score,score-HT,localTeam-SHT,awayTeam-SHT
0,Fr,05.08.2022,20:30,Eintracht Frankfurt,FC Bayern München,1:6,1,6,0:5,0,5


In [141]:
    
# Generate 'NewField' based on comparison of 'Field1' and 'Field2'
df['points-Local'] = df.apply(lambda row: 3 if row['localTeam-Score'] > row['awayTeam-Score'] else (1 if row['localTeam-Score'] == row['awayTeam-Score'] else 0), axis=1)
df['points-Away'] = df.apply(lambda row: 3 if row['awayTeam-Score'] > row['localTeam-Score'] else (1 if row['localTeam-Score'] == row['awayTeam-Score'] else 0), axis=1)

# Reorder columns
df = df[['DoW', 'date', 'time', 'localTeam', 'points-Local', 'awayTeam', 'points-Away', 'score', 'localTeam-Score', 'awayTeam-Score', 'score-HT', 'localTeam-SHT', 'awayTeam-SHT']]

In [142]:
df

,DoW,date,time,localTeam,points-Local,awayTeam,points-Away,score,localTeam-Score,awayTeam-Score,score-HT,localTeam-SHT,awayTeam-SHT
0,Fr,05.08.2022,20:30,Eintracht Frankfurt,0,FC Bayern München,3,1:6,1,6,0:5,0,5
1,Sa,06.08.2022,15:30,1. FC Union Berlin,3,Hertha BSC,0,3:1,3,1,1:0,1,0
2,Sa,06.08.2022,15:30,Bor. Mönchengladbach,3,TSG Hoffenheim,0,3:1,3,1,1:1,1,1
3,Sa,06.08.2022,15:30,FC Augsburg,0,SC Freiburg,3,0:4,0,4,0:0,0,0
4,Sa,06.08.2022,15:30,VfL Bochum,0,FSV Mainz 05,3,1:2,1,2,1:1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,Sa,27.05.2023,15:30,Eintracht Frankfurt,0,SC Freiburg,0,:,NaN,NaN,:,(,)
302,Sa,27.05.2023,15:30,Rasenballsport Leipzig,0,FC Schalke 04,0,:,NaN,NaN,:,(,)
303,Sa,27.05.2023,15:30,VfB Stuttgart,0,TSG Hoffenheim,0,:,NaN,NaN,:,(,)
304,Sa,27.05.2023,15:30,VfL Bochum,0,Bayer 04 Leverkusen,0,:,NaN,NaN,:,(,)


In [143]:
blTable_local = df.groupby('localTeam')['points-Local'].sum()

# blTable_local.sort_values(ascending=False)
blTable_local


localTeam
1. FC Köln                11
1. FC Union Berlin        17
Bayer 04 Leverkusen       11
Bor. Mönchengladbach      18
Borussia Dortmund         16
Eintracht Frankfurt       13
FC Augsburg                5
FC Bayern München         17
FC Schalke 04              7
FSV Mainz 05               7
Hertha BSC                10
Rasenballsport Leipzig    19
SC Freiburg               16
TSG Hoffenheim            10
VfB Stuttgart             11
VfL Bochum                10
VfL Wolfsburg             12
Werder Bremen             10
Name: points-Local, dtype: int64

In [144]:
blTable_away = df.groupby('awayTeam')['points-Away'].sum()

# blTable_away.sort_values(ascending=False)
blTable_away

awayTeam
1. FC Köln                 6
1. FC Union Berlin        10
Bayer 04 Leverkusen        7
Bor. Mönchengladbach       4
Borussia Dortmund          9
Eintracht Frankfurt       14
FC Augsburg               10
FC Bayern München         17
FC Schalke 04              2
FSV Mainz 05              12
Hertha BSC                 4
Rasenballsport Leipzig     9
SC Freiburg               14
TSG Hoffenheim             8
VfB Stuttgart              3
VfL Bochum                 3
VfL Wolfsburg             11
Werder Bremen             11
Name: points-Away, dtype: int64

In [145]:
merge_df = pd.merge(blTable_local, blTable_away, how='outer', left_on='localTeam', right_on='awayTeam')

In [146]:
concat_df = pd.concat([blTable_local, blTable_away], axis=0)

In [147]:
concat_df = concat_df.to_frame()

In [148]:
concat_df.info()
concat_df

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 1. FC Köln to  Werder Bremen
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       36 non-null     int64
dtypes: int64(1)
memory usage: 576.0+ bytes


,0
1. FC Köln,11
1. FC Union Berlin,17
Bayer 04 Leverkusen,11
Bor. Mönchengladbach,18
Borussia Dortmund,16
Eintracht Frankfurt,13
FC Augsburg,5
FC Bayern München,17
FC Schalke 04,7
FSV Mainz 05,7


In [149]:
concat_df.shape

(36, 1)

In [150]:
concat_df.groupby(concat_df.index).sum()

,0
1. FC Köln,6
1. FC Union Berlin,10
Bayer 04 Leverkusen,7
Bor. Mönchengladbach,4
Borussia Dortmund,9
Eintracht Frankfurt,14
FC Augsburg,10
FC Bayern München,17
FC Schalke 04,2
FSV Mainz 05,12


In [151]:
concat_df.reset_index(inplace=True)

In [152]:
concat_df = concat_df.rename(columns={'index': 'Team'})

In [169]:
concat_df = concat_df.rename(columns={0: 'Points' })

In [170]:
concat_df.columns

Index(['Team', 'Points'], dtype='object')

In [178]:
pivot_df = concat_df.pivot_table(index='Team', values='Points', aggfunc='sum').reset_index()

print(pivot_df)

                       Team  Points
0                1. FC Köln       6
1        1. FC Union Berlin      10
2       Bayer 04 Leverkusen       7
3      Bor. Mönchengladbach       4
4         Borussia Dortmund       9
5       Eintracht Frankfurt      14
6               FC Augsburg      10
7         FC Bayern München      17
8             FC Schalke 04       2
9              FSV Mainz 05      12
10               Hertha BSC       4
11   Rasenballsport Leipzig       9
12              SC Freiburg      14
13           TSG Hoffenheim       8
14            VfB Stuttgart       3
15               VfL Bochum       3
16            VfL Wolfsburg      11
17            Werder Bremen      11
18               1. FC Köln      11
19       1. FC Union Berlin      17
20      Bayer 04 Leverkusen      11
21     Bor. Mönchengladbach      18
22        Borussia Dortmund      16
23      Eintracht Frankfurt      13
24              FC Augsburg       5
25        FC Bayern München      17
26            FC Schalke 04 

In [186]:
# Remove duplicate entries for the same team
unique_teams_df = concat_df.drop_duplicates(subset=['Team'])

# Group by the 'Team' column and sum the 'Points'
unique_teams_df['Team']= unique_teams_df['Team'].str.strip()
grouped_df = unique_teams_df.groupby('Team')['Points'].sum().reset_index().sort_values('Points', ascending=False)


grouped_df


,Team,Points
7,FC Bayern München,34
12,SC Freiburg,30
11,Rasenballsport Leipzig,28
1,1. FC Union Berlin,27
5,Eintracht Frankfurt,27
4,Borussia Dortmund,25
16,VfL Wolfsburg,23
3,Bor. Mönchengladbach,22
17,Werder Bremen,21
9,FSV Mainz 05,19


In [176]:
concat_df.groupby('Team')['Points'].sum().reset_index()


,Team,Points
0,1. FC Köln,6
1,1. FC Union Berlin,10
2,Bayer 04 Leverkusen,7
3,Bor. Mönchengladbach,4
4,Borussia Dortmund,9
5,Eintracht Frankfurt,14
6,FC Augsburg,10
7,FC Bayern München,17
8,FC Schalke 04,2
9,FSV Mainz 05,12


In [154]:
merge_df

,points-Local,points-Away
0,NaN,6.0
1,NaN,10.0
2,NaN,7.0
3,NaN,4.0
4,NaN,9.0
5,NaN,14.0
6,NaN,10.0
7,NaN,17.0
8,NaN,2.0
9,NaN,12.0


In [155]:
blTable_local

localTeam
1. FC Köln                11
1. FC Union Berlin        17
Bayer 04 Leverkusen       11
Bor. Mönchengladbach      18
Borussia Dortmund         16
Eintracht Frankfurt       13
FC Augsburg                5
FC Bayern München         17
FC Schalke 04              7
FSV Mainz 05               7
Hertha BSC                10
Rasenballsport Leipzig    19
SC Freiburg               16
TSG Hoffenheim            10
VfB Stuttgart             11
VfL Bochum                10
VfL Wolfsburg             12
Werder Bremen             10
Name: points-Local, dtype: int64

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DoW              306 non-null    object
 1   date             306 non-null    object
 2   time             306 non-null    object
 3   localTeam        306 non-null    object
 4   points-Local     306 non-null    int64 
 5   awayTeam         306 non-null    object
 6   points-Away      306 non-null    int64 
 7   score            306 non-null    object
 8   localTeam-Score  135 non-null    object
 9   awayTeam-Score   135 non-null    object
 10  score-HT         306 non-null    object
 11  localTeam-SHT    306 non-null    object
 12  awayTeam-SHT     306 non-null    object
dtypes: int64(2), object(11)
memory usage: 31.2+ KB


In [157]:
# blTable_local.merge(blTable_away, left_on='lkey')
points_table_df = pd.concat([blTable_local, blTable_away], axis=1)

In [158]:
# points_table = points_table_df.groupby['points-Away'].sum()
points_table_df.index.name = 'Team'
points_table_df

,points-Local,points-Away
Team,,
1. FC Köln,11.0,NaN
1. FC Union Berlin,17.0,NaN
Bayer 04 Leverkusen,11.0,NaN
Bor. Mönchengladbach,18.0,NaN
Borussia Dortmund,16.0,NaN
Eintracht Frankfurt,13.0,NaN
FC Augsburg,5.0,NaN
FC Bayern München,17.0,NaN
FC Schalke 04,7.0,NaN


In [159]:
# Fill NaN values with 0
points_table_df.fillna(0, inplace=True)

# Sum points across both 'points-Local' and 'points-Away' columns
points_table_df['Total Points'] = points_table_df['points-Local'] + points_table_df['points-Away']

# Reset index to make 'Team' a regular column
points_table_df.reset_index(inplace=True)
points_table_df


,Team,points-Local,points-Away,Total Points
0,1. FC Köln,11.0,0.0,11.0
1,1. FC Union Berlin,17.0,0.0,17.0
2,Bayer 04 Leverkusen,11.0,0.0,11.0
3,Bor. Mönchengladbach,18.0,0.0,18.0
4,Borussia Dortmund,16.0,0.0,16.0
5,Eintracht Frankfurt,13.0,0.0,13.0
6,FC Augsburg,5.0,0.0,5.0
7,FC Bayern München,17.0,0.0,17.0
8,FC Schalke 04,7.0,0.0,7.0
9,FSV Mainz 05,7.0,0.0,7.0


In [160]:
bundesliga_Table_df = points_table_df.groupby('Team')['Total Points'].sum().astype(int).to_frame()
# bundesliga_Table.info()
# # If you want to reset the index and make 'Team' a regular column, you can use the reset_index() method:
bundesliga_Table_df.reset_index(inplace=True)

bundesliga_Table_df.sort_values(by=['Total Points'], ascending=False)
bundesliga_Table_df['Team']
# bundesliga_Table_df = bundesliga_Table_df.groupby('Team')['Total Points'].sum()
# bundesliga_Table_df

0                  1. FC Köln
1          1. FC Union Berlin
2         Bayer 04 Leverkusen
3        Bor. Mönchengladbach
4           Borussia Dortmund
5         Eintracht Frankfurt
6                 FC Augsburg
7           FC Bayern München
8               FC Schalke 04
9                FSV Mainz 05
10                 Hertha BSC
11     Rasenballsport Leipzig
12                SC Freiburg
13             TSG Hoffenheim
14              VfB Stuttgart
15                 VfL Bochum
16              VfL Wolfsburg
17              Werder Bremen
18                 1. FC Köln
19         1. FC Union Berlin
20        Bayer 04 Leverkusen
21       Bor. Mönchengladbach
22          Borussia Dortmund
23        Eintracht Frankfurt
24                FC Augsburg
25          FC Bayern München
26              FC Schalke 04
27               FSV Mainz 05
28                 Hertha BSC
29     Rasenballsport Leipzig
30                SC Freiburg
31             TSG Hoffenheim
32              VfB Stuttgart
33        

In [161]:
points_table_df = points_table_df.groupby('Team').sum().reset_index()
points_table_df
bundesliga_Table_df = points_table_df[['Team', 'Total Points']]
bundesliga_Table_df

,Team,Total Points
0,1. FC Köln,6.0
1,1. FC Union Berlin,10.0
2,Bayer 04 Leverkusen,7.0
3,Bor. Mönchengladbach,4.0
4,Borussia Dortmund,9.0
5,Eintracht Frankfurt,14.0
6,FC Augsburg,10.0
7,FC Bayern München,17.0
8,FC Schalke 04,2.0
9,FSV Mainz 05,12.0


In [162]:
bundesliga_Table_df['Team'].unique()


array([' 1. FC Köln', ' 1. FC Union Berlin', ' Bayer 04 Leverkusen',
       ' Bor. Mönchengladbach', ' Borussia Dortmund',
       ' Eintracht Frankfurt', ' FC Augsburg', ' FC Bayern München',
       ' FC Schalke 04', ' FSV Mainz 05', ' Hertha BSC',
       ' Rasenballsport Leipzig', ' SC Freiburg', ' TSG Hoffenheim',
       ' VfB Stuttgart', ' VfL Bochum', ' VfL Wolfsburg',
       ' Werder Bremen', '1. FC Köln', '1. FC Union Berlin',
       'Bayer 04 Leverkusen', 'Bor. Mönchengladbach', 'Borussia Dortmund',
       'Eintracht Frankfurt', 'FC Augsburg', 'FC Bayern München',
       'FC Schalke 04', 'FSV Mainz 05', 'Hertha BSC',
       'Rasenballsport Leipzig', 'SC Freiburg', 'TSG Hoffenheim',
       'VfB Stuttgart', 'VfL Bochum', 'VfL Wolfsburg', 'Werder Bremen'],
      dtype=object)